# Homework 9:

Repeat the procedure we did in class to map the cities of the US using a map and the results of a PCoA, but using a different country of your choosing.

Tips:

These are the packages that you will need:

```
import geopy.distance
import pandas as pd
import plotly.express as px
import skbio
```

You will also need the `world_cities.csv` file to get the city names and coordinates for your country.

1. Choose your country.
2. Get only the cities of that country from the `world_cities.csv` file.
3. Sort the cities by population and select the 20 to 50 most populated cities.
4. Create a data frame with only these cities.
5. Calculate the distance between the cities using their latitude/longitude data from your data frame (geopy).
6. Create a data frame with a table of all the distances between each paur of cities. This needs to be in a distance matrix format, just like we imported from the `cities.csv` file for the US.
7. Perform the PCoA using skbio.
8. Plot the results of the PCoA using plotly express.
9. Plot the map of the country with the cities using the latitude/longitude from your table using plotly express.
10. That's it!

In [4]:
import geopy.distance
import pandas as pd
import plotly.express as px
import skbio

# Load world cities data
world_cities = pd.read_csv('world_cities.csv')

# 1. Choose your country
country = 'France'  # Example: France

# 2. Get only the cities of that country
country_cities = world_cities[world_cities['country'] == country]

# 3. Sort the cities by population and select the 20 to 50 most populated cities
sorted_cities = country_cities.sort_values(by='population', ascending=False)
selected_cities = sorted_cities.iloc[20:50]  # Adjust range as needed

# 4. Create a data frame with only these cities
cities_df = selected_cities[['city', 'lat', 'lng']].reset_index(drop=True)

# 5. Calculate the distance between the cities using their latitude/longitude data
distances = pd.DataFrame(index=cities_df['city'], columns=cities_df['city'])

for city1 in cities_df.itertuples():
    for city2 in cities_df.itertuples():
        coord1 = (city1.lat, city1.lng)
        coord2 = (city2.lat, city2.lng)
        distances.loc[city1.city, city2.city] = geopy.distance.geodesic(coord1, coord2).km

# 6. Create a data frame with a table of all the distances between each pair of cities
# Convert distances to a distance matrix format
distance_matrix = skbio.DistanceMatrix(distances.values, ids=distances.index)

# 7. Perform the PCoA using skbio
pcoa_results = skbio.stats.ordination.pcoa(distance_matrix)

# 8. Plot the results of the PCoA using plotly express
pcoa_df = pd.DataFrame(pcoa_results.samples, columns=['PC1', 'PC2'], index=distances.index)
pcoa_fig = px.scatter(pcoa_df, x='PC1', y='PC2', text=pcoa_df.index, title='PCoA of Cities in ' + country)

# 9. Plot the map of the country with the cities using the latitude/longitude
map_fig = px.scatter_mapbox(cities_df, lat='lat', lon='lng', hover_name='city', zoom=4,
                            title='Cities in ' + country, height=600)
map_fig.update_layout(mapbox_style="carto-positron")

# 10. Display plots
pcoa_fig.show()
map_fig.show()


ModuleNotFoundError: No module named 'geopy'